In [19]:
import os
import numpy as np
from hmmlearn import hmm
from sklearn.externals import joblib
from dtw import dtw
from numpy import linalg as la
import warnings
from sklearn.metrics import confusion_matrix
warnings.filterwarnings('ignore')
path='/Users/lay/Desktop/DHG2016/'
import shutil
np.set_printoptions(threshold=np.inf)  

In [20]:
#set the gesture name, subject list and position list here
#the state_list is the hyperparamter to be tuned for HMM, num_train is the number of train subjects and the number of
#test subjects=number of subjects-number of train subjects
gesture_list=['1','3','4','5','6']
subject_list=[str(i) for i in range(1,21)]
position_list=[str(i) for i in range(1,6)]
single_finger_list=['2','4','5','6']
state_list=[3,3,3,3,3]
num_train=19
testgesture_list=['1','3','4','5','6']
feature_file='feature_0.txt'

In [21]:
#define the subject for training and for test
c=[]
ytrue=[]
ypred=[]
for ite in range(1,21):
    #clear buffer every time
    shutil.rmtree('/Users/lay/Desktop/Jupyter code/buffer') 
    os.mkdir('/Users/lay/Desktop/Jupyter code/buffer')
    import random
    testsubject_list=[str(ite)]
    trainsubject_list=set(subject_list)-set(testsubject_list)
    trainsubject_list=list(trainsubject_list)
    trl=[int(i) for i in trainsubject_list]
    trl.sort()
    trainsubject_list=[str(i) for i in trl]
    gesture_dict={}
    length_dict={}
    for gesture in gesture_list:
        feature=[]
        length=[]
        for subject in trainsubject_list:
            for position in position_list:
                if gesture in single_finger_list:
                    finger='1'
                else:
                    finger='2'
                new_path=path+'gesture_'+gesture+'/finger_'+finger+'/subject_'+subject+'/essai_'+position+'/'
                f=open(new_path+feature_file)
                iter_f=iter(f)
                L=[]
                flag=0
                for line in iter_f:
                    l=line.split()
                    l=[float(i) for i in l]
                    L.append(l)
                    flag=flag+1
                feature.extend(L)
                length.append(flag)    
        gesture_dict[gesture]=feature
        length_dict[gesture]=length   

    #training 
    flag=1
    from hmmlearn import hmm
    for gesture in gesture_list:
        X=np.array(gesture_dict[gesture])
        le=length_dict[gesture]
        remodel=hmm.GaussianHMM(n_components=state_list[flag-1],n_iter=100,init_params="cm", params="cmt")
        remodel.startprob_ = np.array([1.0, 0.0, 0.0])
        remodel.transmat_ = np.array([[0.5, 0.5, 0.0],
                                [0.0, 0.5, 0.5],
                                 [0.0, 0.0, 1.0]])
        remodel.fit(X,le)
        #print(remodel.monitor_.converged)
        joblib.dump(remodel,'/Users/lay/Desktop/Jupyter code/buffer/'+'nc_hmm'+str(flag)+'.pkl')
        flag=flag+1

    ##test 
    num=0
    correct=0
    for gesture in testgesture_list:
        for subject in testsubject_list:
            for position in position_list:
                if gesture in single_finger_list:
                    finger='1'
                else:
                    finger='2'
                num=num+1
                new_path=path+'gesture_'+gesture+'/finger_'+finger+'/subject_'+subject+'/essai_'+position+'/'
                f=open(new_path+feature_file)
                iter_f=iter(f)
                L=[]
                for line in iter_f:
                    l=line.split()
                    l=[float(i) for i in l]
                    L.append(l)
                Y=np.array(L)
                pre=[]
                for j in range(len(gesture_list)):
                    hmm=joblib.load('/Users/lay/Desktop/Jupyter code/buffer/'+'nc_hmm'+str(j+1)+'.pkl')
                    pre.append(hmm.score(Y))   
                iid=pre.index((max(pre)))
                ypred.append(iid)
                ytrue.append(gesture_list.index(gesture))
                if iid==gesture_list.index(gesture):
                    correct=correct+1
    print('iter'+ str(ite)+'correct rate is '+str(correct/num))
    c.append(correct/num)
print('average correct rate is'+ str(np.mean(c)))
confusion_matrix=confusion_matrix(ytrue, ypred)

iter1correct rate is 1.0
iter2correct rate is 0.52
iter3correct rate is 0.76
iter4correct rate is 0.88
iter5correct rate is 0.64
iter6correct rate is 0.72
iter7correct rate is 0.64
iter8correct rate is 0.6
iter9correct rate is 0.72
iter10correct rate is 0.84
iter11correct rate is 0.84
iter12correct rate is 0.72
iter13correct rate is 0.6
iter14correct rate is 0.76
iter15correct rate is 1.0
iter16correct rate is 0.64
iter17correct rate is 0.76
iter18correct rate is 0.84
iter19correct rate is 0.64
iter20correct rate is 0.76
average correct rate is0.744


In [23]:
X.shape

(3071, 256)

In [6]:
confusion_matrix

array([[79,  6,  9,  3,  3],
       [ 6, 81, 10,  2,  1],
       [ 5,  6, 47, 22, 20],
       [ 6,  0,  9, 75, 10],
       [ 3,  4, 19, 17, 57]])

In [9]:
shutil.rmtree('/Users/lay/Desktop/Jupyter code/buffer') 
os.mkdir('/Users/lay/Desktop/Jupyter code/buffer')